In [4]:
!pip install pyarrow zstandard


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [5]:
import pyarrow.feather as feather
import zstandard

import pyarrow.feather as feather
# CONTENT BASED EMBEDDINGS
# Specify the path to your .feather.zstd file
file_path = '/content/drive/Shareddrives/CSCE 470 Project/reddit.feather.zstd'

# Read the Feather file using pyarrow (which automatically handles decompression)
table = feather.read_table(file_path)

# Convert the table to a pandas DataFrame for easier viewing
all_reddit_embeds = table.to_pandas()

# Print or manipulate the DataFrame as needed
print(all_reddit_embeds)
# James Charles UCucot-Zp428OwkyRm2I7v2Q
c = all_reddit_embeds[all_reddit_embeds['channelId'] == 'UCucot-Zp428OwkyRm2I7v2Q']
# there is no embedding for james charles

                      channelId         0         1         2         3  \
0      UC--3c8RqSfAqYBdDjIG3UNA  0.205217 -0.106509 -0.168593  0.304357   
1      UC--3ozenTzry39-xMqAPQlw  0.049721 -0.063819 -0.122901  0.066127   
2      UC--3pN-wtQrC48GAK8wsBsg  0.147761 -0.102652 -0.055768  0.215948   
3      UC--70ql_IxJmhmqXqrkJrWQ  0.120708  0.003842 -0.170499  0.249080   
4      UC--EwQJeJ6SKlJdswehQH4g  0.100566  0.010194 -0.121284  0.028830   
...                         ...       ...       ...       ...       ...   
43769  UCzzamhkOmpSnZ7BSBikvKnA  0.036068 -0.055506  0.018135  0.141581   
43770  UCzzfu83LhPMMuhtDVA75rfw  0.149524  0.007451 -0.068932  0.149564   
43771  UCzzsltdnAgTMSlPOvjko_Dw  0.121148  0.006482 -0.207436  0.242270   
43772  UCzztaOtjmaZoiLvWLagq7bg  0.106945 -0.077044 -0.245714  0.167454   
43773  UCzzzUN8yvD2LRAnY-lhzyLQ -0.012686 -0.124965 -0.113191  0.132516   

              4         5         6         7         8  ...       140  \
0     -0.117221 -0.015534

In [11]:
#extract embeds of only our top 1000

chan_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chanIDs.csv'

chan_ids = pd.read_csv(chan_file_path)

merged_df = pd.merge(all_reddit_embeds, chan_ids, left_on='channelId', right_on='Youtube Url', how='inner')

# Drop the redundant column
merged_df = merged_df.drop(columns='Youtube Url')

matching_chan_df = merged_df['channelId']

In [7]:
from sklearn.metrics.pairwise import cosine_similarity


In [8]:

# Function to get recommendations based on cosine similarity
def get_channel_recommendations(channel_id, embeddings_df, top_n=5):
    # Find the index of the given channel
    channel_index = embeddings_df.index[embeddings_df['channelId'] == channel_id].tolist()[0]

    # Select the embeddings for the given channel
    channel_embeddings = embeddings_df.iloc[channel_index, 1:]  # Assuming embeddings start from the second column

    # Reshape the embeddings to a 2D array
    channel_embeddings = channel_embeddings.values.reshape(1, -1)

    # Calculate cosine similarity between the given channel and all other channels
    similarity_scores = cosine_similarity(channel_embeddings, embeddings_df.iloc[:, 1:])

    # Get indices of top_n most similar channels (excluding the channel itself)
    similar_channel_indices = similarity_scores.argsort()[0][-top_n-1:-1][::-1]

    # Get the channel_id of recommended channels
    recommended_channels = embeddings_df['channelId'].iloc[similar_channel_indices].tolist()

    return recommended_channels



In [23]:

# good test case is ColdplayVEVO UCDPM_n1atn2ijUwHd0NNRQw
chan_recommendations_dict = {}
for chan in matching_chan_df:
  recommendations = get_channel_recommendations(chan, merged_df)
  chan_recommendations_dict[chan] = recommendations
chan_recommendations_dict

{'UC-0D1RlSzcvBoODCKSoLl2g': ['UCCdYYruzqbr0FdNehbUoUiw',
  'UC5p_l5ZeB_wGjO_yDXwiqvw',
  'UCvlE5gTbOvjiolFlEm-c_Ow',
  'UCJplp5SjeGSdVdwsfb9Q7lQ',
  'UCGmnsW623G1r-Chmo5RB4Yw'],
 'UC-2Y8dQb0S6DtpxNgAKoJKA': ['UCKy1dAqELo0zrOtPkf0eTMw',
  'UCpqXJOEqGS-TCnazcHCo0rA',
  'UCClNRixXlagwAd--5MwJKCw',
  'UCHdMK5Ef2El8KbD3L_WgANg',
  'UCfLuMSIDmeWRYpuCQL0OJ6A'],
 'UC-Eh1lR_tWVtv9AvxLybMtQ': ['UCtylTUUVIGY_i5afsQYeBZA',
  'UC-yXuc1__OzjwpsJPlxYUCQ',
  'UCByOQJjav0CUDwxCk-jVNRQ',
  'UCyFZMEnm1il5Wv3a6tPscbA',
  'UCeLHszkByNZtPKcaVXOCOQQ'],
 'UC-SV8-bUJfXjrRMnp7F8Wzw': ['UCPDis9pjXuqyI7RYLJ-TTSA',
  'UCUtZaxDF3hD5VK4xRYFBePQ',
  'UCkAGrHCLFmlK3H2kd6isipg',
  'UCd1fLoVFooPeWqCEYVUJZqg',
  'UCGt7X90Au6BV8rf49BiM6Dg'],
 'UC-lHJZR3Gqxm24_Vd_AJ5Yw': ['UCGmnsW623G1r-Chmo5RB4Yw',
  'UC4-79UOlP48-QNGgCko5p2g',
  'UCilwZiBBfI9X6yiZRzWty8Q',
  'UCMNmwqCtCSpftrbvR3KkHDA',
  'UChGJGhZ9SOOHvBB0Y4DOO_w'],
 'UC-yXuc1__OzjwpsJPlxYUCQ': ['UC-Eh1lR_tWVtv9AvxLybMtQ',
  'UCtylTUUVIGY_i5afsQYeBZA',
  'UClW4jraMKz6Qj

In [39]:
# correlate chan ids to chan names
chan_names_file_path = '/content/drive/Shareddrives/CSCE 470 Project/1000_chan_names_IDs.csv'

chan_names_ids = pd.read_csv(chan_names_file_path)

chan_usernames_dict = {}
# Traverse the chan_recommendations_dict and correlate IDs to names
for chan_id, recommendations in chan_recommendations_dict.items():
    # Find the corresponding username for the channel ID
    matching_row = chan_names_ids[chan_names_ids['Youtube Url'] == chan_id]
    matching_row = matching_row['username'].values[0]
    chan_usernames_dict[matching_row] = []
    for rec in recommendations:
      #translate the recs too
      translated_rec = chan_names_ids[chan_names_ids['Youtube Url'] == rec]
      chan_usernames_dict[matching_row].append(translated_rec['username'].values[0])

# Now chan_usernames_dict contains the mapping of channel IDs to usernames
print(chan_usernames_dict)


{'Emily Tube': ['Taylor RS', 'TommyInnit', 'Vlad and Niki', 'Like Nastya', 'KSIOlajidebtHD'], 'PlayStation': ['IGNentertainment', 'theRadBrad', 'H2ODelirious', 'Slogomanify', 'kwebbelkop'], 'TygaTygaTV': ['GooodLifeFilms', 'WorldStarHipHopTV', 'DrakeOfficial', 'RapGeniusVideo', 'postmalone'], 'RomanAtwoodVlogs': ['failarmy', 'BritainsGotTalent09', 'MrBean', 'MegatoadStonie', 'RayWilliamJohnson'], 'PewDiePie': ['KSIOlajidebtHD', 'MrBeast Shorts', 'oRugrat', 'aynakoitsroi', "Ryan's World"], 'WorldStarHipHopTV': ['TygaTygaTV', 'GooodLifeFilms', 'YoungBoy Never Broke Again', 'DrakeOfficial', 'RapGeniusVideo'], 'Crafty Panda': ['gottalentglobal', 'onemeeeliondollars', 'barcroftmedia', 'MrBean', 'failarmy'], 'DJ Khaled': ['MattyBRaps', 'Cardi B', 'JakePaulProductions', 'postmalone', 'RapGeniusVideo'], 'Doctor Mike': ['BuzzFeedVideo', 'BRIGHT SIDE', 'buzzfeedblue', 'TEDEducation', 'AsapSCIENCE'], 'xoxxxoooxo': ['RapGeniusVideo', 'GooodLifeFilms', 'VEVO', 'TygaTygaTV', 'postmalone'], 'SaraBeau

In [40]:
df_channels_usernames = pd.DataFrame(list(chan_usernames_dict.items()), columns=['Channel', 'Recommendations'])

# Save the DataFrame to a CSV file
csv_file_path = 'reddit_recs.csv'
df_channels_usernames.to_csv(csv_file_path, index=False)
